In [8]:
import pandas as pd

In [9]:
st = pd.read_csv("data_stores.csv")

In [12]:
las_vegas = st.loc[st["City"]=="Las Vegas",["Longitude","Latitude"]]

In [13]:
las_vegas

,Longitude,Latitude
20644,-115.20,36.12
20645,-115.15,36.04
20646,-115.17,36.13
20647,-115.28,36.22
20648,-115.18,36.11
...,...,...
20796,-115.18,36.10
20797,-115.21,36.24
20798,-115.30,36.11
20799,-115.17,36.09
